# Preprocess

In [10]:
import numpy as np
import pandas as pd

In [11]:
train_data = pd.read_csv("/content/drive/MyDrive/Semester7/ML/project/data/train-split.csv", index_col=0)
train_data.reset_index(inplace = True, drop = True)
train_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,label
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,0


In [12]:
test_data = pd.read_csv("/content/drive/MyDrive/Semester7/ML/project/data/test-split.csv", index_col=0)
test_data.reset_index(inplace = True, drop = True)

submission_data = pd.read_csv("/content/drive/MyDrive/Semester7/ML/project/data/pump-test.csv", index_col=0)
submission_data.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


# Categorical features


In [13]:
def calc_percentage(df, feature):
  return (df[feature].value_counts()/len(df) > 0.01)

def reduce_categories(row, percentage, feature):
  if row[feature] not in percentage.keys() or percentage[row[feature]] == False:
    row[feature] = "other"
  return row

In [14]:
cat_cols = ["funder", "installer", "wpt_name", "basin", "lga", "scheme_name" , "quantity", "extraction_type_class", "scheme_management", "management", "payment", "source", "water_quality",  "waterpoint_type", "subvillage"]

for feature in cat_cols:
  print(feature)
  mod_val = train_data[feature].mode()[0]
  train_data[feature] = train_data[feature].fillna(mod_val)
  test_data[feature] = test_data[feature].fillna(mod_val)
  submission_data[feature] = submission_data[feature].fillna(mod_val)

  train_data[feature].replace("0", "other", inplace = True)
  test_data[feature].replace("0", "other", inplace = True)
  submission_data[feature].replace("0", "other", inplace = True)

  percentage = calc_percentage(train_data, feature)
  train_data = train_data.apply(reduce_categories, args = (percentage, feature), axis='columns')
  test_data = test_data.apply(reduce_categories, args = (percentage, feature), axis='columns')
  submission_data = submission_data.apply(reduce_categories, args = (percentage, feature), axis='columns')

cat_cols2 = ["permit", "public_meeting"]

for feature in cat_cols2:
  print(feature)
  mod = train_data[feature].mode()[0]
  train_data[feature] = train_data[feature].fillna(mod)
  test_data[feature] = test_data[feature].fillna(mod)
  submission_data[feature] = submission_data[feature].fillna(mod)

  train_data[feature].replace("0", "other", inplace = True)
  test_data[feature].replace("0", "other", inplace = True)
  submission_data[feature].replace("0", "other", inplace = True)


funder
installer
wpt_name
basin
lga
scheme_name
quantity
extraction_type_class
scheme_management
management
payment
source
water_quality
waterpoint_type
subvillage
permit
public_meeting


In [15]:
cat_train_data = train_data.copy()
cat_test_data = test_data.copy()
cat_submission_data = submission_data.copy()

# Cont features

In [16]:
train_data = cat_train_data.copy()
test_data = cat_test_data.copy()
submission_data = cat_submission_data.copy()

In [17]:
# adjust test data accordingly
temp = train_data.copy()
temp_test = test_data.copy()
temp_sub = submission_data.copy()

amount_dict = {}
long_dict = {}
lat_dict = {}
pop_dict = {}
const_dict = {}

def replace(group, mean, mini, maxi, mean_dict):
    outliers = ((group <=mini) | (group > maxi))
    mean = group[~outliers].mean() if len(group[~outliers])>0 else mean
    group[outliers] = mean
    mean_dict[group.name] = mean
    return group

def replace_test(group, mean, mini, maxi, mean_dict):
    outliers = ((group <=mini) | (group > maxi))
    mean = mean_dict[group.name] if (group.name in mean_dict.keys()) else mean
    group[outliers] = mean
    return group


amount_tsh_mean = temp[(temp.amount_tsh >0) & (temp.amount_tsh < 100)].amount_tsh.mean()
temp.amount_tsh = temp.groupby('subvillage').amount_tsh.transform(replace, amount_tsh_mean, 0, 100, amount_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)

long_mean = temp[(temp.longitude >25) & (temp.longitude < 45)].longitude.mean()
temp.longitude = temp.groupby('subvillage').longitude.transform(replace, long_mean, 25, 45, long_dict)
temp_test.longitude = temp_test.groupby('subvillage').longitude.transform(replace_test, long_mean, 25, 45, long_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)

lat_mean = temp[(temp.latitude >-15) & (temp.latitude < 0)].latitude.mean()
temp.latitude = temp.groupby('subvillage').latitude.transform(replace, lat_mean, -15, -.0001, lat_dict)
temp_test.latitude = temp_test.groupby('subvillage').latitude.transform(replace_test, lat_mean, -15, -.0001, lat_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)

pop_mean = temp[(temp.population >0) & (temp.population < 20000)].population.mean()
temp.population = temp.groupby('subvillage').population.transform(replace, pop_mean, 0, 20000, pop_dict)
temp_test.population = temp_test.groupby('subvillage').population.transform(replace_test, pop_mean, 0, 20000, pop_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)

const_mean = temp[(temp.construction_year > 1400) & (temp.construction_year < 2021)].construction_year.mean()
temp.construction_year = temp.groupby('funder').construction_year.transform(replace, const_mean, 1400, 2021, const_dict)
temp_test.construction_year = temp_test.groupby('funder').construction_year.transform(replace_test, const_mean, 1400, 2021, const_dict)
temp_test.amount_tsh = temp_test.groupby('subvillage').amount_tsh.transform(replace_test, amount_tsh_mean, 0, 100, amount_dict)


In [18]:
def replace(group, test_group):
    groupgt0 = group[group>0]
    q1 = groupgt0.quantile(0.25)
    q3 = groupgt0.quantile(0.75)
    iqr = q3 - q1
    median = groupgt0.median()
    outliers = (group< q1-1.5*iqr) | (group> q3+1.5*iqr)
    test_outliers = (test_group < q1-1.5*iqr) | (test_group > q3+1.5*iqr)
    group[outliers] = median
    test_group[test_outliers] = median
    train_dict = {"q1":q1, "q3": q3, "iqr": iqr, "median": median}
    return group, test_group


In [19]:
train_data = temp
test_data = temp_test
submission_data = temp_sub

train_data.construction_year = 2021- train_data.construction_year
test_data.construction_year = 2021- test_data.construction_year
submission_data.construction_year = 2021- submission_data.construction_year

# Split

In [20]:
features = ["amount_tsh",	"funder",	"gps_height",	"installer", "longitude",	"latitude",	"wpt_name",	"basin",	"region_code",	"district_code",	"lga",	"population",	"scheme_management",	"scheme_name",	"permit",	"construction_year",	"extraction_type_class",	"management",	"payment",	"water_quality",	"quantity",	"source",	"waterpoint_type", "subvillage"]

X_train = train_data[features]
y_train = train_data["label"]
X_test = test_data[features]
y_test = test_data[["id", "label"]]

submission_data = submission_data[features]

In [21]:
y_test.head()
X_test.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region_code,district_code,lga,population,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,subvillage
0,37.866417,other,0,other,32.711100,-5.146712,other,Lake Tanganyika,14,6,other,279.968324,VWC,K,True,20.628583,handpump,vwc,never pay,salty,seasonal,machine dbh,hand pump,other
1,20.000000,Government Of Tanzania,1007,DWE,36.806079,-3.478559,other,Pangani,2,2,Arusha Rural,100.000000,VWC,other,True,21.000000,gravity,vwc,pay per bucket,soft,insufficient,spring,communal standpipe,other
2,6.000000,other,962,DWE,37.479436,-3.368198,other,Pangani,3,4,Moshi Rural,45.000000,Water Board,other,True,13.000000,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe,other
3,37.866417,Government Of Tanzania,1415,DWE,34.621598,-5.173136,other,Internal,13,2,Singida Rural,1.000000,VWC,other,True,51.000000,motorpump,vwc,unknown,unknown,dry,machine dbh,communal standpipe,other
4,37.866417,Government Of Tanzania,0,DWE,34.462228,-8.575780,other,Rufiji,12,7,Mbarali,279.968324,VWC,K,False,32.105643,gravity,vwc,never pay,soft,insufficient,river,communal standpipe,other


# Encode

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

X_train2 = X_train.copy()
X_test2 = X_test.copy()
sub_data2 = submission_data.copy()

enc_type = 2
object_cols = ["funder", "installer", "wpt_name", "basin", "lga", "scheme_name", "quantity", "extraction_type_class", "scheme_management", "management", "payment", "source", "water_quality",  "waterpoint_type", "subvillage"]

if(enc_type == 1):

  OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse=False)

  OH_encoder.fit(X_train2[object_cols])

  OH_cols_train = pd.DataFrame(OH_encoder.transform(X_train2[object_cols]))
  OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test2[object_cols]))
  OH_cols_sub = pd.DataFrame(OH_encoder.transform(X_test2[object_cols]))

  # One-hot encoding removed index; put it back
  OH_cols_train.index = X_train2.index
  OH_cols_test.index = X_test2.index
  OH_cols_sub.index = sub_data2.index

  # Remove categorical columns (will replace with one-hot encoding)
  num_X_train = X_train2.drop(object_cols, axis=1)
  num_X_test = X_test2.drop(object_cols, axis=1)
  num_sub_data = sub_data2.drop(object_cols, axis=1)

  # Add one-hot encoded columns to numerical features
  X_train2 = pd.concat([num_X_train, OH_cols_train], axis=1)
  X_test2 = pd.concat([num_X_test, OH_cols_test], axis=1)
  sub_data2 = pd.concat([num_sub_data, OH_cols_sub], axis=1)

else:
  OD_encoder = OrdinalEncoder()

  X_train2.loc[:, object_cols] = OD_encoder.fit_transform(X_train2.loc[:, object_cols])
  X_test2.loc[:, object_cols] = OD_encoder.fit_transform(X_test2.loc[:, object_cols])
  sub_data2.loc[:, object_cols] = OD_encoder.fit_transform(sub_data2.loc[:, object_cols])

X_train2.head()


,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region_code,district_code,lga,population,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,subvillage
0,37.866417,16.0,1390,10.0,34.938093,-9.856322,2.0,1.0,11,5,35.0,109.000000,4.0,2.0,False,22.000000,0.0,4.0,2.0,3.0,1.0,8.0,0.0,1.0
1,37.866417,16.0,1399,10.0,34.698766,-2.147466,1.0,4.0,20,2,31.0,280.000000,1.0,0.0,True,11.000000,0.0,8.0,0.0,3.0,2.0,5.0,0.0,1.0
2,25.000000,16.0,686,10.0,37.460664,-3.821329,3.0,5.0,21,4,35.0,250.000000,4.0,2.0,True,12.000000,0.0,4.0,4.0,3.0,1.0,0.0,1.0,1.0
3,37.866417,13.0,263,10.0,38.486161,-11.155298,3.0,7.0,90,63,35.0,58.000000,4.0,0.0,True,35.000000,4.0,4.0,0.0,3.0,0.0,3.0,1.0,1.0
4,37.866417,16.0,0,10.0,31.130847,-1.825359,0.0,4.0,18,1,6.0,279.968324,4.0,0.0,True,20.628583,0.0,1.0,0.0,3.0,3.0,5.0,0.0,1.0


In [23]:
X_test2.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region_code,district_code,lga,population,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,subvillage
0,37.866417,16.0,0,10.0,32.711100,-5.146712,3.0,3.0,14,6,35.0,279.968324,4.0,0.0,True,20.628583,1.0,4.0,0.0,2.0,3.0,3.0,2.0,1.0
1,20.000000,5.0,1007,4.0,36.806079,-3.478559,3.0,5.0,2,2,0.0,100.000000,4.0,2.0,True,21.000000,0.0,4.0,4.0,3.0,2.0,8.0,0.0,1.0
2,6.000000,16.0,962,4.0,37.479436,-3.368198,3.0,5.0,3,4,22.0,45.000000,7.0,2.0,True,13.000000,0.0,6.0,4.0,3.0,1.0,8.0,0.0,1.0
3,37.866417,5.0,1415,4.0,34.621598,-5.173136,3.0,0.0,13,2,32.0,1.000000,4.0,2.0,True,51.000000,2.0,4.0,6.0,4.0,0.0,3.0,0.0,1.0
4,37.866417,5.0,0,4.0,34.462228,-8.575780,3.0,6.0,12,7,18.0,279.968324,4.0,0.0,False,32.105643,0.0,4.0,0.0,3.0,2.0,6.0,0.0,1.0


# Model

In [24]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=1)

In [25]:
model.fit(X_train2, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=100, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [26]:
val_preds = model.predict(X_test2)
val_preds=np.rint(val_preds).astype(int)
val_preds = pd.DataFrame({"id": test_data.id, "label":val_preds})
val_preds.head()

,id,label
0,53934,1
1,1843,1
2,74197,0
3,51072,2
4,17386,2


In [27]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test["label"], val_preds["label"])
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)   

print("- recall: ", recall)
print("- precision: ", precision)                                                                                                                     

accuracy = y_test[(y_test["label"] == val_preds["label"])].count()/len(y_test)
accuracy[0]
print(f"- Accuracy:{accuracy[0]}")

- recall:  [0.72873991 0.5473251  0.67258883]
- precision:  [0.85010862 0.16583541 0.93200469]
- Accuracy:0.6923583662714098


In [28]:
sub_data2.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region_code,district_code,lga,population,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,subvillage
id,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,16.0,1996,10.0,35.290799,-4.059696,3.0,0.0,21,3,35.0,321,2.0,0.0,True,9,3.0,2.0,0.0,3.0,3.0,5.0,4.0,1.0
51630,0.0,5.0,1569,4.0,36.656709,-3.309214,3.0,5.0,2,2,0.0,300,4.0,2.0,True,21,0.0,4.0,0.0,3.0,2.0,8.0,0.0,1.0
17168,0.0,5.0,1567,4.0,34.767863,-5.004344,3.0,0.0,13,2,32.0,500,4.0,2.0,True,11,3.0,4.0,0.0,3.0,2.0,5.0,4.0,1.0
45559,0.0,16.0,267,10.0,38.058046,-9.418672,3.0,7.0,80,43,35.0,250,4.0,0.0,True,34,3.0,4.0,6.0,3.0,0.0,7.0,4.0,1.0
49871,500.0,16.0,1260,10.0,35.006123,-10.950412,3.0,7.0,10,3,19.0,60,7.0,2.0,True,21,0.0,6.0,3.0,3.0,1.0,8.0,0.0,1.0


In [29]:
print(train_data["label"].value_counts())
print(y_test["label"].value_counts())
print(val_preds["label"].value_counts())

0    30648
2    21642
1     4074
Name: label, dtype: int64
0    1611
2    1182
1     243
Name: label, dtype: int64
0    1381
2     853
1     802
Name: label, dtype: int64


In [30]:
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
feature_importances

[('quantity', 0.18),
 ('longitude', 0.15),
 ('latitude', 0.15),
 ('waterpoint_type', 0.09),
 ('gps_height', 0.07),
 ('construction_year', 0.07),
 ('population', 0.05),
 ('extraction_type_class', 0.03),
 ('payment', 0.03),
 ('funder', 0.02),
 ('region_code', 0.02),
 ('district_code', 0.02),
 ('lga', 0.02),
 ('management', 0.02),
 ('source', 0.02),
 ('amount_tsh', 0.01),
 ('installer', 0.01),
 ('wpt_name', 0.01),
 ('basin', 0.01),
 ('scheme_management', 0.01),
 ('scheme_name', 0.01),
 ('permit', 0.01),
 ('water_quality', 0.01),
 ('subvillage', 0.0)]

# Output

In [32]:
predictions = model.predict(sub_data2)
predictions=np.rint(predictions).astype(int)

labels = ['functional', 'functional needs repair', 'non functional']

output = pd.DataFrame({"id":submission_data.index, "status_group": predictions})

def change(value):
  if value == 0:
    return "functional"
  if value == 1:
    return "functional needs repair"
  return "non functional"

output.status_group = output.status_group.transform(change)

output.head()

,id,status_group
0,50785,functional needs repair
1,51630,functional
2,17168,functional needs repair
3,45559,non functional
4,49871,functional


In [33]:
output.to_csv("/content/drive/MyDrive/Semester7/ML/project/data/out.csv", index = False)